In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'e:\\Github repositories\\end-to-end-fake-news-detection'

In [58]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [59]:
from FakeNewsDetection.utils.common import read_yaml, create_directory
from FakeNewsDetection.constants import *

class ConfigurationManager:
    def __init__(self, 
                 config_path= CONFIG_FILE,
                 parama_path= PARAMS_FILE):
        
        self.config = read_yaml(config_path)
        self.params = read_yaml(parama_path)

        create_directory([self.config.artifact_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directory([config.root_dir])

        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )


In [60]:
import os
import urllib.request as request
import zipfile
from FakeNewsDetection import logger
import gdown

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    # function to download data from google drive
    
    def download_data(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                logger.info("Downloading data from source")
                # download data from google drive
                gdown.download(self.config.source_url, self.config.local_data_file, quiet=False)
                logger.info("Data downloaded successfully")
            else:
                logger.info("Data already exists")
        except Exception as e:
            logger.error(f"Error downloading data: {e}")
            raise e
    def unzip_data(self):
        try:
            if os.path.exists(self.config.local_data_file):
                with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                    zip_ref.extractall(self.config.unzip_dir)
                logger.info("Data unzipped successfully")
            else:
                logger.info("Data already unzipped")
        except Exception as e:
            logger.error(f"Error unzipping data: {e}")
            raise e


In [64]:
if __name__ == '__main__':
    try:
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(data_ingestion_config)
        data_ingestion.download_data()
        data_ingestion.unzip_data()
    except Exception as e:  
        logger.error(f"Error in data ingestion: {e}")
        raise e


[2024-11-21 09:26:34,952] [INFO] [common.py:26] [Loaded yaml file from config\config.yaml]
[2024-11-21 09:26:34,958] [INFO] [common.py:26] [Loaded yaml file from params.yaml]
[2024-11-21 09:26:34,960] [INFO] [common.py:51] [Created directory artifacts]
[2024-11-21 09:26:34,963] [INFO] [common.py:51] [Created directory artifacts/data_ingestion]
[2024-11-21 09:26:34,965] [INFO] [274604666.py:21] [Data already exists]
[2024-11-21 09:26:34,971] [INFO] [274604666.py:30] [Data unzipped successfully]


UsageError: Line magic function `%unzip` not found.
